## Imports

In [2]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta

## Config

In [2]:
RAW_BASE = "../datasets/circuit-of-the-americas/COTA/Race 1"
OUTPUT_ROOT = os.path.abspath("../datasets_clean")
os.makedirs(OUTPUT_ROOT, exist_ok=True)

ANALYSIS_FILE = os.path.join(RAW_BASE, "23_AnalysisEnduranceWithSections_Race 1_Anonymized.csv")
WEATHER_FILE = os.path.join(RAW_BASE, "26_Weather_Race 1_Anonymized.csv")
TELEMETRY_FILE = os.path.join(RAW_BASE, "R1_cota_telemetry_data.csv")

LAP_START_FILE = os.path.join(RAW_BASE, "COTA_lap_start_time_R1.csv")
LAP_END_FILE = os.path.join(RAW_BASE, "COTA_lap_end_time_R1.csv")
LAP_TIME_FILE = os.path.join(RAW_BASE, "COTA_lap_time_R1.csv")

## Utility

In [3]:
def time_to_seconds(t):
    """Convert strings like 'M:SS.mmm' or 'H:MM:SS.mmm' or 'MM:SS' to seconds (float).
    Returns NaN for missing or unparsable inputs."""
    if pd.isna(t):
        return np.nan
    s = str(t).strip()
    if s == '':
        return np.nan
    # replace comma decimal separators
    s = s.replace(',', '.')
    parts = s.split(':')
    try:
        if len(parts) == 1:
            return float(parts[0])
        if len(parts) == 2:
            m, sec = parts
            return float(m) * 60.0 + float(sec)
        if len(parts) == 3:
            h, m, sec = parts
            return float(h) * 3600.0 + float(m) * 60.0 + float(sec)
    except Exception:
        return np.nan

def clean_column_names(df):
    df = df.copy()
    df.columns = [c.strip() for c in df.columns]
    return df

## Cleaning time

In [4]:
# ---------------------- 1) Load & clean "analysis with sections" ----------------------
print("Loading analysis file:", ANALYSIS_FILE)
analysis_df = pd.read_csv(ANALYSIS_FILE, sep=';', dtype=str)
analysis_df = clean_column_names(analysis_df)
print("Raw columns:", analysis_df.columns.tolist())

# Drop fully-empty columns
analysis_df = analysis_df.dropna(how='all', axis=1)

# Drop unwanted columns to make files readable in Excel
for drop_col in ['CLASS', 'GROUP', 'MANUFACTURER']:
    if drop_col in analysis_df.columns:
        analysis_df = analysis_df.drop(columns=[drop_col])

# Convert numeric-ish columns
for col in ['NUMBER', 'DRIVER_NUMBER', 'LAP_NUMBER', 'KPH', 'TOP_SPEED', 'PIT_TIME']:
    if col in analysis_df.columns:
        analysis_df[col] = pd.to_numeric(analysis_df[col], errors='coerce')

# Convert LAP times and sector strings to seconds
for col in ['LAP_TIME', 'S1', 'S2', 'S3', 'S1_SECONDS', 'S2_SECONDS', 'S3_SECONDS']:
    if col in analysis_df.columns:
        # prefer S?_SECONDS if present numeric; otherwise convert textual S1 etc.
        if col.endswith('_SECONDS'):
            analysis_df[col] = pd.to_numeric(analysis_df[col], errors='coerce')
        else:
            analysis_df[col + '_SEC'] = analysis_df[col].apply(time_to_seconds)

# If explicit seconds columns exist but not the _SEC ones, copy them
for s_col in ['S1_SECONDS', 'S2_SECONDS', 'S3_SECONDS']:
    if s_col in analysis_df.columns and (s_col.replace('_SECONDS', '') + '_SEC') not in analysis_df.columns:
        analysis_df[s_col.replace('_SECONDS', '') + '_SEC'] = pd.to_numeric(analysis_df[s_col], errors='coerce')

# Normalize flag column
if 'FLAG_AT_FL' in analysis_df.columns:
    analysis_df['FLAG_AT_FL'] = analysis_df['FLAG_AT_FL'].astype(str).str.strip()

# Create a numeric Lap_Time if not present
if 'LAP_TIME_SEC' not in analysis_df.columns and 'LAP_TIME' in analysis_df.columns:
    analysis_df['LAP_TIME_SEC'] = analysis_df['LAP_TIME'].apply(time_to_seconds)

print("Loaded analysis rows:", len(analysis_df))

# ---------------------- 2) Filter out unwanted laps ----------------------
# Remove laps under FCY/SC/AUTO flags and laps that finished in pit
bad_flags = set(['FCY', 'SC', 'YELLOW'])
if 'FLAG_AT_FL' in analysis_df.columns:
    before = len(analysis_df)
    analysis_df = analysis_df[~analysis_df['FLAG_AT_FL'].isin(bad_flags)]
    print(f"Filtered by flag: {before} -> {len(analysis_df)}")

# Proper pit filtering: 'B' marks a pit entry lap, '0' means normal lap
if 'CROSSING_FINISH_LINE_IN_PIT' in analysis_df.columns:
    before = len(analysis_df)
    analysis_df['CROSSING_FINISH_LINE_IN_PIT'] = analysis_df['CROSSING_FINISH_LINE_IN_PIT'].astype(str).str.strip()
    analysis_df = analysis_df[analysis_df['CROSSING_FINISH_LINE_IN_PIT'] != 'B']
    print(f"Filtered pit-crossing laps: {before} -> {len(analysis_df)}")

# Remove clearly invalid lap times
if 'LAP_TIME_SEC' in analysis_df.columns:
    before = len(analysis_df)
    analysis_df = analysis_df[pd.to_numeric(analysis_df['LAP_TIME_SEC'], errors='coerce') > 5.0] # remove zeros and extremely small
    analysis_df = analysis_df[pd.to_numeric(analysis_df['LAP_TIME_SEC'], errors='coerce') < 3600.0] # sanity upper bound
    print(f"Filtered by lap time sanity: {before} -> {len(analysis_df)}")

# ---------------------- 3) Save cleaned per-driver CSVs (compact columns) ----------------------
# Select a useful subset of columns for driver-coaching CSVs
default_cols = [
'NUMBER','DRIVER_NUMBER','LAP_NUMBER','LAP_TIME','LAP_TIME_SEC',
'S1','S1_SEC','S2','S2_SEC','S3','S3_SEC',
'KPH','TOP_SPEED','ELAPSED','HOUR','FLAG_AT_FL'
]
cols_to_save = [c for c in default_cols if c in analysis_df.columns]
print('Columns saved per driver:', cols_to_save)

# Ensure NUMBER is numeric for filename safety
if 'NUMBER' in analysis_df.columns:
    analysis_df['NUMBER'] = pd.to_numeric(analysis_df['NUMBER'], errors='coerce')

for num, grp in analysis_df.groupby('NUMBER'):
    safe_num = int(num) if not pd.isna(num) else 'unknown'
    out_path = os.path.join(OUTPUT_ROOT, f"driver_{safe_num}.csv")
    grp[cols_to_save].to_csv(out_path, index=False, sep=',')
    print(f"Saved driver file: {out_path} ({len(grp)} laps)")

# ---------------------- 4) Load telemetry (if present) and pivot into wide form ----------------------
if os.path.exists(TELEMETRY_FILE):
    print('Loading telemetry (this may be large):', TELEMETRY_FILE)
    tel_df = pd.read_csv(TELEMETRY_FILE)
    tel_df = clean_column_names(tel_df)
    # Parse times
    for tcol in ['meta_time','timestamp']:
        if tcol in tel_df.columns:
            tel_df[tcol] = pd.to_datetime(tel_df[tcol], errors='coerce', utc=True)
    # Ensure values numeric when possible
    tel_df['telemetry_value'] = pd.to_numeric(tel_df['telemetry_value'], errors='coerce')

    # Pivot: make each telemetry_name a column per vehicle_id + timestamp
    index_cols = [c for c in ['vehicle_id','meta_time','timestamp','lap'] if c in tel_df.columns]
    print('Telemetry index cols:', index_cols)
    tel_wide = tel_df.pivot_table(index=index_cols, columns='telemetry_name', values='telemetry_value', aggfunc='first').reset_index()
    print('Telemetry wide shape:', tel_wide.shape)

    # Save per-vehicle telemetry sampled per-lap timestamp rows (if lap info exists)
    if 'vehicle_id' in tel_wide.columns:
        telem_out = os.path.join(OUTPUT_ROOT, 'telemetry_per_timestamp.csv')
        tel_wide.to_csv(telem_out, index=False)
        print('Saved telemetry per-timestamp wide file to:', telem_out)
else:
    print('No telemetry file found at', TELEMETRY_FILE)

# ---------------------- 5) Compute per-lap telemetry summaries (vectorized) ----------------------
print("Generating per-lap and per-sector telemetry summary for MAD-filtered laps...")

if os.path.exists(TELEMETRY_FILE):
    tel_df = pd.read_csv(TELEMETRY_FILE)
    tel_df = clean_column_names(tel_df)
    tel_df['timestamp'] = pd.to_datetime(tel_df['timestamp'], errors='coerce', utc=True)
    tel_df['vehicle_id'] = tel_df['vehicle_id'].astype(str)
    tel_df = tel_df.dropna(subset=['timestamp', 'telemetry_value'])
    tel_df['telemetry_value'] = pd.to_numeric(tel_df['telemetry_value'], errors='coerce')

    # Keep only valid laps from driver_session_stats
    driver_stats_path = os.path.join(OUTPUT_ROOT, "driver_session_stats.csv")
    if os.path.exists(driver_stats_path):
        valid_laps_df = pd.read_csv(driver_stats_path)
        valid_pairs = analysis_df.merge(
            valid_laps_df[['DriverNumber']],
            left_on='NUMBER', right_on='DriverNumber',
            how='inner'
        )
        valid_laps = set(zip(valid_pairs['NUMBER'].astype(str), valid_pairs['LAP_NUMBER']))
        tel_df = tel_df[tel_df.apply(lambda r: (r['vehicle_id'], r.get('lap', np.nan)) in valid_laps, axis=1)]

    # Compute per-lap, per-sector metrics
    per_lap_list = []
    for (vid, lap), group in tel_df.groupby(['vehicle_id','lap']):
        lap_metrics = {'vehicle_id': vid, 'lap': lap}

        # Whole-lap metrics
        lap_metrics['mean_throttle'] = group.loc[group['telemetry_name'].isin(['ath','aps','throttle']),'telemetry_value'].mean()
        lap_metrics['mean_brake'] = group.loc[group['telemetry_name'].isin(['pbrake_f','pbrake_r','brake']),'telemetry_value'].mean()
        sa = group.loc[group['telemetry_name']=='Steering_Angle','telemetry_value']
        lap_metrics['steering_smoothness'] = sa.diff().abs().mean() if not sa.empty else np.nan

        # Optional: sector metrics if sector telemetry columns exist
        for sector in ['S1','S2','S3']:
            sector_mask = group['telemetry_name'].str.contains(sector, case=False)
            if sector_mask.any():
                lap_metrics[f'{sector}_mean_throttle'] = group.loc[group['telemetry_name'].isin(['ath','aps','throttle']) & sector_mask,'telemetry_value'].mean()
                lap_metrics[f'{sector}_mean_brake'] = group.loc[group['telemetry_name'].isin(['pbrake_f','pbrake_r','brake']) & sector_mask,'telemetry_value'].mean()
                sa_sector = group.loc[sector_mask & (group['telemetry_name']=='Steering_Angle'),'telemetry_value']
                lap_metrics[f'{sector}_steering_smoothness'] = sa_sector.diff().abs().mean() if not sa_sector.empty else np.nan

        per_lap_list.append(lap_metrics)

    if per_lap_list:
        per_lap_df = pd.DataFrame(per_lap_list)
        per_lap_path = os.path.join(OUTPUT_ROOT, 'per_lap_telemetry_summary.csv')
        per_lap_df.to_csv(per_lap_path, index=False)
        print('✅ Saved per-lap telemetry summary to:', per_lap_path)
else:
    print('⚠️ No telemetry data matched any valid MAD-filtered laps.')

# ---------------------- 6) Quick driver session statistics (filter extreme laps) ----------------------
print("\nComputing driver session statistics with sanity filtering...")

# Ensure numeric
for col in ['LAP_TIME_SEC', 'S1_SEC', 'S2_SEC', 'S3_SEC']:
    if col in analysis_df.columns:
        analysis_df[col] = pd.to_numeric(analysis_df[col], errors='coerce')

# Filter obviously invalid or extreme laps (more than 15 sec slower than driver mean)
if 'NUMBER' in analysis_df.columns:
    filtered_df_list = []
    for driver, grp in analysis_df.groupby('NUMBER'):
        grp = grp.copy()
        median = grp['LAP_TIME_SEC'].median()
        mad = (grp['LAP_TIME_SEC'] - median).abs().median()
        cutoff = median + 3 * mad  # 3 MADs above median

        # Debug info
        print(f"\nDriver {driver}:")
        print(f"  Number of laps before filtering: {len(grp)}")
        print(f"  Median lap time: {median:.3f} sec")
        print(f"  MAD (Median Abs Deviation): {mad:.3f} sec")
        print(f"  Cutoff for valid laps: {cutoff:.3f} sec")

        # Apply filtering
        filtered_grp = grp[grp['LAP_TIME_SEC'] <= cutoff]
        print(f"  Number of laps after filtering: {len(filtered_grp)}")
        if len(filtered_grp) < len(grp):
            removed = len(grp) - len(filtered_grp)
            print(f"  → Removed {removed} lap(s) as outliers.")

        filtered_df_list.append(filtered_grp)

    valid_laps_df = pd.concat(filtered_df_list, ignore_index=True)
else:
    valid_laps_df = analysis_df.copy()

# Compute per-driver stats
agg_dict = {
    'LAP_TIME_SEC': ['count','min','mean','std'],
    'S1_SEC': 'min', 'S2_SEC': 'min', 'S3_SEC': 'min'
}
driver_stats_df = valid_laps_df.groupby('NUMBER', as_index=False).agg(agg_dict)

# Flatten columns
driver_stats_df.columns = [
    'DriverNumber','Laps','BestLap(s)','AvgLap(s)','StdDev(s)',
    'S1Best','S2Best','S3Best'
]
driver_stats_df['TheoreticalBest(s)'] = driver_stats_df[['S1Best','S2Best','S3Best']].sum(axis=1)

# Round
driver_stats_df[['BestLap(s)','AvgLap(s)','StdDev(s)','TheoreticalBest(s)']] = \
    driver_stats_df[['BestLap(s)','AvgLap(s)','StdDev(s)','TheoreticalBest(s)']].round(3)

# Save CSV
driver_stats_path = os.path.join(OUTPUT_ROOT, "driver_session_stats.csv")
driver_stats_df.to_csv(driver_stats_path, index=False)
print(f"✅ Saved driver session stats to: {driver_stats_path}")
display(driver_stats_df)

Loading analysis file: ../datasets/circuit-of-the-americas/COTA/Race 1\23_AnalysisEnduranceWithSections_Race 1_Anonymized.csv
Raw columns: ['NUMBER', 'DRIVER_NUMBER', 'LAP_NUMBER', 'LAP_TIME', 'LAP_IMPROVEMENT', 'CROSSING_FINISH_LINE_IN_PIT', 'S1', 'S1_IMPROVEMENT', 'S2', 'S2_IMPROVEMENT', 'S3', 'S3_IMPROVEMENT', 'KPH', 'ELAPSED', 'HOUR', 'S1_LARGE', 'S2_LARGE', 'S3_LARGE', 'TOP_SPEED', 'PIT_TIME', 'CLASS', 'GROUP', 'MANUFACTURER', 'FLAG_AT_FL', 'S1_SECONDS', 'S2_SECONDS', 'S3_SECONDS', 'IM1a_time', 'IM1a_elapsed', 'IM1_time', 'IM1_elapsed', 'IM2a_time', 'IM2a_elapsed', 'IM2_time', 'IM2_elapsed', 'IM3a_time', 'IM3a_elapsed', 'FL_time', 'FL_elapsed']
Loaded analysis rows: 486
Filtered by flag: 486 -> 397
Filtered pit-crossing laps: 397 -> 389
Filtered by lap time sanity: 389 -> 389
Columns saved per driver: ['NUMBER', 'DRIVER_NUMBER', 'LAP_NUMBER', 'LAP_TIME', 'LAP_TIME_SEC', 'S1', 'S1_SEC', 'S2', 'S2_SEC', 'S3', 'S3_SEC', 'KPH', 'TOP_SPEED', 'ELAPSED', 'HOUR', 'FLAG_AT_FL']
Saved drive

,DriverNumber,Laps,BestLap(s),AvgLap(s),StdDev(s),S1Best,S2Best,S3Best,TheoreticalBest(s)
0,3,23,151.035,152.509,0.889,32.914,57.593,60.037,150.544
1,5,12,150.848,152.119,1.002,32.758,57.820,59.768,150.346
2,7,12,148.678,149.221,0.450,32.544,56.689,59.129,148.362
3,8,11,153.790,155.357,1.061,33.467,58.681,61.245,153.393
4,11,8,151.702,152.245,0.399,33.048,57.988,60.261,151.297
5,13,13,148.745,149.498,0.688,32.526,56.786,59.131,148.443
6,14,12,150.363,151.495,0.932,32.837,57.505,59.468,149.810
7,15,3,151.899,152.039,0.140,33.127,58.301,60.320,151.748
8,16,12,148.726,149.288,0.464,32.431,56.909,59.189,148.529
9,18,11,153.543,155.986,1.495,33.551,58.470,61.343,153.364


In [5]:
# ---------------------- Filter extreme laps using MAD per driver ----------------------
if 'NUMBER' in analysis_df.columns:
    filtered_df_list = []
    driver_filter_summary = []  # keep record for debugging/inspection

    for driver, grp in analysis_df.groupby('NUMBER'):
        grp = grp.copy()
        median = grp['LAP_TIME_SEC'].median()
        mad = (grp['LAP_TIME_SEC'] - median).abs().median()
        cutoff = median + 3 * mad  # 3 MADs above median

        # Filter outliers
        filtered_grp = grp[grp['LAP_TIME_SEC'] <= cutoff]

        # Collect summary info
        driver_filter_summary.append({
            'DriverNumber': driver,
            'TotalLaps': len(grp),
            'ValidLaps': len(filtered_grp),
            'RemovedLaps': len(grp) - len(filtered_grp),
            'MedianLap(s)': median,
            'MAD(s)': mad,
            'Cutoff(s)': cutoff
        })

        print(f"\nDriver {driver}:")
        print(f"  Total laps: {len(grp)} | Valid: {len(filtered_grp)} | Removed: {len(grp) - len(filtered_grp)}")
        print(f"  Median: {median:.3f}s | MAD: {mad:.3f}s | Cutoff: {cutoff:.3f}s")

        # Save filtered laps to individual CSV
        safe_num = int(driver) if not pd.isna(driver) else 'unknown'
        out_path = os.path.join(OUTPUT_ROOT, f"driver_{safe_num}.csv")
        filtered_grp.to_csv(out_path, index=False)
        print(f"  ✅ Saved filtered driver file: {out_path} ({len(filtered_grp)} valid laps)")

        filtered_df_list.append(filtered_grp)

    # Combine all valid laps for session-level stats
    valid_laps_df = pd.concat(filtered_df_list, ignore_index=True)

    # Optional summary overview
    filter_summary_df = pd.DataFrame(driver_filter_summary)
    print("\nSummary of filtering across drivers:")
    display(filter_summary_df)
else:
    valid_laps_df = analysis_df.copy()

# ---------------------- Compute per-driver statistics (only valid laps) ----------------------
agg_dict = {
    'LAP_TIME_SEC': ['count','min','mean','std'],
    'S1_SEC': 'min', 'S2_SEC': 'min', 'S3_SEC': 'min'
}

driver_stats_df = valid_laps_df.groupby('NUMBER', as_index=False).agg(agg_dict)
driver_stats_df.columns = [
    'DriverNumber','Laps','BestLap(s)','AvgLap(s)','StdDev(s)',
    'S1Best','S2Best','S3Best'
]
driver_stats_df['TheoreticalBest(s)'] = driver_stats_df[['S1Best','S2Best','S3Best']].sum(axis=1)

# Round for readability
driver_stats_df[['BestLap(s)','AvgLap(s)','StdDev(s)','TheoreticalBest(s)']] = \
    driver_stats_df[['BestLap(s)','AvgLap(s)','StdDev(s)','TheoreticalBest(s)']].round(3)

# Save updated session-level stats
driver_stats_path = os.path.join(OUTPUT_ROOT, "driver_session_stats.csv")
driver_stats_df.to_csv(driver_stats_path, index=False)
print(f"\n✅ Saved filtered driver session stats to: {driver_stats_path}")
display(driver_stats_df)



Driver 3:
  Total laps: 28 | Valid: 23 | Removed: 5
  Median: 152.644s | MAD: 0.939s | Cutoff: 155.461s
  ✅ Saved filtered driver file: c:\Users\jacks\OneDrive\Desktop\progamming projects\toyota gr analysis\datasets_clean\driver_3.csv (23 valid laps)

Driver 5:
  Total laps: 14 | Valid: 12 | Removed: 2
  Median: 152.252s | MAD: 1.034s | Cutoff: 155.354s
  ✅ Saved filtered driver file: c:\Users\jacks\OneDrive\Desktop\progamming projects\toyota gr analysis\datasets_clean\driver_5.csv (12 valid laps)

Driver 7:
  Total laps: 14 | Valid: 12 | Removed: 2
  Median: 149.195s | MAD: 0.386s | Cutoff: 150.353s
  ✅ Saved filtered driver file: c:\Users\jacks\OneDrive\Desktop\progamming projects\toyota gr analysis\datasets_clean\driver_7.csv (12 valid laps)

Driver 8:
  Total laps: 14 | Valid: 11 | Removed: 3
  Median: 155.683s | MAD: 1.055s | Cutoff: 158.846s
  ✅ Saved filtered driver file: c:\Users\jacks\OneDrive\Desktop\progamming projects\toyota gr analysis\datasets_clean\driver_8.csv (11 vali

,DriverNumber,TotalLaps,ValidLaps,RemovedLaps,MedianLap(s),MAD(s),Cutoff(s)
0,3,28,23,5,152.6440,0.9390,155.4610
1,5,14,12,2,152.2520,1.0340,155.3540
2,7,14,12,2,149.1950,0.3860,150.3530
3,8,14,11,3,155.6825,1.0545,158.8460
4,11,9,8,1,152.2870,0.3150,153.2320
5,13,14,13,1,149.4515,0.5805,151.1930
6,14,14,12,2,151.4475,0.8145,153.8910
7,15,4,3,1,152.1095,0.1395,152.5280
8,16,14,12,2,149.2720,0.3725,150.3895
9,18,14,11,3,156.5590,1.7525,161.8165



✅ Saved filtered driver session stats to: c:\Users\jacks\OneDrive\Desktop\progamming projects\toyota gr analysis\datasets_clean\driver_session_stats.csv


,DriverNumber,Laps,BestLap(s),AvgLap(s),StdDev(s),S1Best,S2Best,S3Best,TheoreticalBest(s)
0,3,23,151.035,152.509,0.889,32.914,57.593,60.037,150.544
1,5,12,150.848,152.119,1.002,32.758,57.820,59.768,150.346
2,7,12,148.678,149.221,0.450,32.544,56.689,59.129,148.362
3,8,11,153.790,155.357,1.061,33.467,58.681,61.245,153.393
4,11,8,151.702,152.245,0.399,33.048,57.988,60.261,151.297
5,13,13,148.745,149.498,0.688,32.526,56.786,59.131,148.443
6,14,12,150.363,151.495,0.932,32.837,57.505,59.468,149.810
7,15,3,151.899,152.039,0.140,33.127,58.301,60.320,151.748
8,16,12,148.726,149.288,0.464,32.431,56.909,59.189,148.529
9,18,11,153.543,155.986,1.495,33.551,58.470,61.343,153.364


In [4]:
HARD_CUTOFF = 40
telemetry_file = "../datasets_clean/per_lap_telemetry_summary.csv"

# Load the telemetry summary
telemetry_df = pd.read_csv(telemetry_file)

# Extract driver number from vehicle_id (last number after dash)
telemetry_df['NUMBER'] = telemetry_df['vehicle_id'].str.split('-').str[-1].astype(int)

filtered_list = []

for driver, grp in telemetry_df.groupby('NUMBER'):
    median = grp['mean_throttle'].median()
    mad = (grp['mean_throttle'] - median).abs().median()
    # Define low-throttle cutoff (e.g., median - 3*MAD)
    cutoff = median - 3 * mad
    grp_filtered = grp[(grp['mean_throttle'] >= cutoff) & (grp['mean_throttle'] >= HARD_CUTOFF)]
    filtered_list.append(grp_filtered)
    print(f"Driver {driver}: median={median:.2f}, MAD={mad:.2f}, cutoff={cutoff:.2f}, removed={len(grp)-len(grp_filtered)} laps")

# Concatenate filtered groups
filtered_telemetry_df = pd.concat(filtered_list, ignore_index=True)

# Overwrite the original CSV
filtered_telemetry_df.to_csv(telemetry_file, index=False)

print(f"Filtered telemetry saved. Remaining rows: {len(filtered_telemetry_df)}")

Driver 2: median=20.52, MAD=7.40, cutoff=-1.68, removed=2 laps
Driver 3: median=76.04, MAD=1.36, cutoff=71.95, removed=1 laps
Driver 5: median=75.10, MAD=1.09, cutoff=71.84, removed=0 laps
Driver 7: median=74.92, MAD=0.49, cutoff=73.46, removed=0 laps
Driver 8: median=66.16, MAD=1.47, cutoff=61.74, removed=0 laps
Driver 11: median=70.63, MAD=0.82, cutoff=68.17, removed=0 laps
Driver 13: median=78.00, MAD=0.49, cutoff=76.52, removed=0 laps
Driver 14: median=77.04, MAD=0.91, cutoff=74.32, removed=0 laps
Driver 15: median=74.75, MAD=0.50, cutoff=73.26, removed=0 laps
Driver 16: median=74.27, MAD=0.52, cutoff=72.72, removed=1 laps
Driver 18: median=68.60, MAD=2.24, cutoff=61.87, removed=2 laps
Driver 21: median=74.53, MAD=0.51, cutoff=73.00, removed=2 laps
Driver 31: median=76.09, MAD=0.75, cutoff=73.83, removed=2 laps
Driver 41: median=75.60, MAD=0.69, cutoff=73.53, removed=2 laps
Driver 46: median=78.14, MAD=0.49, cutoff=76.68, removed=0 laps
Driver 47: median=74.08, MAD=0.71, cutoff=71.